The goal here is to load the exgalm5 data for all the 6-bands and restructure them so that we have the median depths for all the filters in one file. Before that, I test whether the saved i-band depth matches that from the OSWG summary excel sheet.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import os
import pandas as pd
%matplotlib inline

In [2]:
import time
import copy
import numpy as np
import healpy as hp
import os

%matplotlib inline

In [3]:
# import plot settings
import sys
sys.path.append('/global/homes/a/awan/my-utils/')
from pyutils import settings

In [4]:
path = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.7_-0.1cuts/exgalm5_data/summary_data/'
#os.listdir(path)

In [5]:
outdir = 'paper-data/exgalm5-related_v1.7/'
order_path = 'paper-data/summary_csv_v1.7_-0.1cuts/'
os.makedirs(outdir, exist_ok=True)

##### read in just the i-band data first

In [6]:
yr_tags = ['y1', 'y3', 'y6', 'y10']

new_data = {}
for yr_tag in yr_tags:
    fname = [f for f in os.listdir(path) if f.__contains__('_i-band_') and f.__contains__('_%s_' % yr_tag) and f.endswith('csv')][0]
    print('reading in %s' % fname)
    new_data[yr_tag] = pd.read_csv('%s/%s' % (path, fname))
    print(yr_tag)

reading in median-depth_i-band_eg-pixels_y1_limi24.65_nside256.csv
y1
reading in median-depth_i-band_eg-pixels_y3_limi25.25_nside256.csv
y3
reading in median-depth_i-band_eg-pixels_y6_limi25.619999999999997_nside256.csv
y6
reading in median-depth_i-band_eg-pixels_y10_limi25.9_nside256.csv
y10


In [7]:
new_data['y3'].dbname.values.shape

(74,)

##### read in the summary data 

In [8]:
#summary_data_old = pd.read_csv('paper-data/summary_csv_v1.7_-0.1cuts--try1/static_metrics.csv')
summary_data = pd.read_csv('paper-data/summary_csv_v1.7_-0.1cuts/static_metrics.csv')
summary_data

,name,median Y1 $i$-band coadded depth in effective survey area,median Y3 $i$-band coadded depth in effective survey area,median Y6 $i$-band coadded depth in effective survey area,median Y10 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area,std in Y3 $i$-band coadded depth in effective survey area,std in Y6 $i$-band coadded depth in effective survey area,std in Y10 $i$-band coadded depth in effective survey area,Y1 effective survey area,Y3 effective survey area,Y6 effective survey area,Y10 effective survey area
0,rolling_scale0.9_nslice3_v1.7_10yrs,25.03,25.65,25.98,26.24,0.17,0.13,0.12,0.11,14779.75,15062.38,15065.69,15079.06
1,rolling_scale0.2_nslice3_v1.7_10yrs,25.03,25.66,25.99,26.26,0.17,0.13,0.12,0.12,14779.75,15298.96,15184.45,15167.56
2,rolling_scale1.0_nslice3_v1.7_10yrs,25.03,25.65,25.98,26.24,0.17,0.13,0.12,0.11,14779.75,14946.19,15061.18,15048.59
3,rolling_scale0.9_nslice2_v1.7_10yrs,25.03,25.64,25.98,26.24,0.17,0.13,0.12,0.12,14779.75,15243.41,15125.33,15080.17
4,rolling_scale0.6_nslice3_v1.7_10yrs,25.03,25.65,25.99,26.25,0.17,0.13,0.12,0.12,14779.75,15224.84,15124.81,15098.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,cadence_drive_gl100_gcbv1.7_10yrs,25.02,25.66,26.01,26.27,0.16,0.13,0.12,0.11,14639.17,15250.07,15193.31,15201.71
70,cadence_drive_gl100v1.7_10yrs,25.01,25.66,26.00,26.27,0.16,0.13,0.12,0.11,14618.87,15245.93,15185.60,15188.75
71,cadence_drive_gl30v1.7_10yrs,25.03,25.67,26.01,26.27,0.16,0.13,0.12,0.11,14617.45,15232.50,15186.23,15191.95
72,cadence_drive_gl200_gcbv1.7_10yrs,24.99,25.62,25.97,26.24,0.17,0.13,0.12,0.12,13883.60,15061.02,15011.50,15003.69


In [9]:
new_data['y3'].keys(), summary_data.keys()

(Index(['dbname', 'Area (deg2)', 'i-band depth: median', 'i-band depth: std'], dtype='object'),
 Index(['name', 'median Y1 $i$-band coadded depth in effective survey area',
        'median Y3 $i$-band coadded depth in effective survey area',
        'median Y6 $i$-band coadded depth in effective survey area',
        'median Y10 $i$-band coadded depth in effective survey area',
        'std in Y1 $i$-band coadded depth in effective survey area',
        'std in Y3 $i$-band coadded depth in effective survey area',
        'std in Y6 $i$-band coadded depth in effective survey area',
        'std in Y10 $i$-band coadded depth in effective survey area',
        'Y1 effective survey area ', 'Y3 effective survey area ',
        'Y6 effective survey area ', 'Y10 effective survey area '],
       dtype='object'))

#### compare the overlapping data (area, i-band depth, depth stddev) between the two files

look to see if all the dbs in the new data are presented in the summary data

In [10]:
kk = 0
for yr_tag in yr_tags:
    print('checking %s' % yr_tag)
    mapper = {'i-band depth: median': 'median Y%s $i$-band coadded depth in effective survey area' % yr_tag.split('y')[-1], 
          'Area (deg2)': 'Y%s effective survey area '  % yr_tag.split('y')[-1],
          'i-band depth: std': 'std in Y%s $i$-band coadded depth in effective survey area' % yr_tag.split('y')[-1]
         }
    
    inds = []
    for i, db in enumerate( new_data[yr_tag]['dbname'] ):
        ind = np.where(summary_data['name'] == '%s' % db)[0]
        
        if len(ind) == 0:
            print('## didnt find %s in the summary data' % db)
        else:
            inds.append(ind[0])
            for stat_key in mapper:
                #print('## "%s" vs. "%s"' % (stat_key, mapper[stat_key]))
                val1 = new_data[yr_tag][stat_key].values[i]
                val2 = float(summary_data[mapper[stat_key]].values[ind][0])
                if val1 != val2:
                    print('things dont match for %s: %s: %s vs. %s' % (db, stat_key, val1, val2))
                    kk += 1
print('## problematic cases: %s' % kk)

checking y1
checking y3
checking y6
checking y10
## problematic cases: 0


look to see if all the dbs in the all-metrics data are presented in the new data

In [11]:
for yr_tag in yr_tags:
    print('\nchecking %s' % yr_tag)
    mapper = {'i-band depth: median': 'median Y%s $i$-band coadded depth in effective survey area' % yr_tag.split('y')[-1], 
          'Area (deg2)': 'Y%s effective survey area '  % yr_tag.split('y')[-1],
          'i-band depth: std': 'std in Y%s $i$-band coadded depth in effective survey area' % yr_tag.split('y')[-1]
         }
    
    inds = []
    for i, db in enumerate( summary_data['name'] ):
        ind = np.where(new_data[yr_tag]['dbname'] == db.split('.db')[0])[0]
        
        if len(ind)  == 0:
            print('%s not in the new data' % db)
        
        inds.append(ind)
        for stat_key in mapper:
            #print('## "%s" vs. "%s"' % (stat_key, mapper[stat_key]))
            val1 = new_data[yr_tag][stat_key].values[ind]
            val2 = float(summary_data[mapper[stat_key]].values[i])
            if val1 != val2:
                print('things dont match for %s: %s: %s vs. %s' % (db, stat_key, val1, val2))


checking y1

checking y3

checking y6

checking y10


confirm that all the dbs are present for all the years.

one more check: number of dbs for each year:

In [12]:
for yr_tag in yr_tags:
    print(yr_tag, len(new_data[yr_tag]['dbname']))

y1 74
y3 74
y6 74
y10 74


okay things match -- good. can proceed to saving things now.

#### assemble the data to save

need to make sure that the dbs for all the bands are added in the same order for all years

In [13]:
bands = ['u', 'g', 'r', 'i', 'z', 'y']

In [14]:
depth_med_all_yrs = {}
depth_stddev_all_yrs = {}

for yr_tag in ['y10', 'y3', 'y6', 'y1']:
    depth_med_this_yr = pd.DataFrame()
    depth_stddev_this_yr = pd.DataFrame()
    
    for i, band in enumerate( bands ):
        dont_add = False
        fname = [f for f in os.listdir(path) if f.__contains__('_%s-band_' % band) \
                                                 and f.__contains__('_%s_' % yr_tag)
                ][0]
        print('reading in %s' % fname)
        data_ = pd.read_csv('%s/%s' % (path, fname))
        #print(data_.dbname.values)
        band_key_median = '%s-band median coadded dust-corrected depth in eg-footprint' % band
        band_key_stddev = '%s-band coadded dust-corrected depth stddev in eg-footprint' % band
        if i == 0:
            # add to the all-data dictionary
            # median depth
            depth_med_this_yr['dbname'] = data_['dbname'].values
            depth_med_this_yr['yr'] = [yr_tag] * len( data_['dbname'].values )
            # depth stddev
            depth_stddev_this_yr['dbname'] = data_['dbname'].values
            depth_stddev_this_yr['yr'] = [yr_tag] * len( data_['dbname'].values )
        else:
            # check if the order is okay
            check = (depth_med_this_yr['dbname'].values != data_['dbname'].values)
            if isinstance(check, np.ndarray):
                check = check.any()
            else: check = check
            if check:
                print('\n## dbname order is different for %s-band vs. previous bands for %s.' % (band, yr_tag) )
                print('reordering ... ')
                
                # reorder things
                data_ = depth_med_this_yr.merge(data_, on='dbname', suffixes=['_prev', '_%s' % yr_tag])
                
                # check to ensure the db order is the same
                print('checking the order again .. should not print anything if the order is now fine.\n')
                check = (depth_med_this_yr['dbname'].values != data_['dbname'].values)
                if isinstance(check, np.ndarray):
                    check = check.any()
                else: check = check
                if check:
                    print('## somethings wrong with the db order')
                    

        depth_med_this_yr[band_key_median] = data_['%s-band depth: median' % band].values
        depth_stddev_this_yr[band_key_stddev] = data_['%s-band depth: std' % band].values

    depth_med_all_yrs[yr_tag] = depth_med_this_yr
    depth_stddev_all_yrs[yr_tag] = depth_stddev_this_yr

reading in median-depth_u-band_eg-pixels_y10_limi25.9_nside256.csv
reading in median-depth_g-band_eg-pixels_y10_limi25.9_nside256.csv
reading in median-depth_r-band_eg-pixels_y10_limi25.9_nside256.csv
reading in median-depth_i-band_eg-pixels_y10_limi25.9_nside256.csv
reading in median-depth_z-band_eg-pixels_y10_limi25.9_nside256.csv
reading in median-depth_y-band_eg-pixels_y10_limi25.9_nside256.csv
reading in median-depth_u-band_eg-pixels_y3_limi25.25_nside256.csv
reading in median-depth_g-band_eg-pixels_y3_limi25.25_nside256.csv
reading in median-depth_r-band_eg-pixels_y3_limi25.25_nside256.csv
reading in median-depth_i-band_eg-pixels_y3_limi25.25_nside256.csv

## dbname order is different for i-band vs. previous bands for y3.
reordering ... 
checking the order again .. should not print anything if the order is now fine.

reading in median-depth_z-band_eg-pixels_y3_limi25.25_nside256.csv
reading in median-depth_y-band_eg-pixels_y3_limi25.25_nside256.csv
reading in median-depth_u-band_

order the dbs for y1, y3, y6 as is for y10

In [15]:
#### assemble data to save first in the order of each years pd df

In [16]:
#y10_order = pd.DataFrame({'dbname': depth_med_all_yrs['y10']['dbname']})
y10_order = []
dbs_order = pd.read_csv('%s/given_order_v1.7.csv' % order_path, )
dbs_order = dbs_order.rename(columns={'name': 'dbname'})
dbs_order

,dbname
0,rolling_scale0.9_nslice3_v1.7_10yrs
1,rolling_scale0.2_nslice3_v1.7_10yrs
2,rolling_scale1.0_nslice3_v1.7_10yrs
3,rolling_scale0.9_nslice2_v1.7_10yrs
4,rolling_scale0.6_nslice3_v1.7_10yrs
...,...
69,cadence_drive_gl100_gcbv1.7_10yrs
70,cadence_drive_gl100v1.7_10yrs
71,cadence_drive_gl30v1.7_10yrs
72,cadence_drive_gl200_gcbv1.7_10yrs


In [17]:
depth_med_all_yrs['y10']

,dbname,yr,u-band median coadded dust-corrected depth in eg-footprint,g-band median coadded dust-corrected depth in eg-footprint,r-band median coadded dust-corrected depth in eg-footprint,i-band median coadded dust-corrected depth in eg-footprint,z-band median coadded dust-corrected depth in eg-footprint,y-band median coadded dust-corrected depth in eg-footprint
0,twi_pairs_repeat_v1.7_10yrs,y10,25.40,26.73,26.78,26.27,25.56,24.81
1,twi_neo_pattern6_v1.7_10yrs,y10,25.39,26.69,26.76,26.23,25.54,24.77
2,rolling_nm_scale0.4_nslice2_v1.7_10yrs,y10,25.42,26.73,26.78,26.26,25.57,24.81
3,footprint_1_v1.710yrs,y10,25.38,26.68,26.76,26.24,25.54,24.78
4,footprint_6_v1.710yrs,y10,25.36,26.68,26.75,26.23,25.53,24.78
...,...,...,...,...,...,...,...,...
69,ddf_dither1.50_v1.7_10yrs,y10,25.41,26.72,26.78,26.27,25.57,24.80
70,ddf_dither1.00_v1.7_10yrs,y10,25.41,26.71,26.79,26.26,25.57,24.80
71,cadence_drive_gl30_gcbv1.7_10yrs,y10,25.40,26.64,26.75,26.27,25.58,24.80
72,rolling_nm_scale0.9_nslice3_v1.7_10yrs,y10,25.41,26.71,26.77,26.25,25.56,24.81


In [18]:
# median depth data - move away from the nested dictionary
depth_data = pd.DataFrame()
for i, yr_tag in enumerate( depth_med_all_yrs ):
    depth_med_all_yrs[yr_tag] = dbs_order.merge(depth_med_all_yrs[yr_tag], on='dbname')
    
    if i == 0:
        depth_data = depth_med_all_yrs[yr_tag]
    else:
        # stack them up
        depth_data = depth_data.append(depth_med_all_yrs[yr_tag])
        
depth_data.reset_index(inplace=True, drop=True)

depth_data

,dbname,yr,u-band median coadded dust-corrected depth in eg-footprint,g-band median coadded dust-corrected depth in eg-footprint,r-band median coadded dust-corrected depth in eg-footprint,i-band median coadded dust-corrected depth in eg-footprint,z-band median coadded dust-corrected depth in eg-footprint,y-band median coadded dust-corrected depth in eg-footprint
0,rolling_scale0.9_nslice3_v1.7_10yrs,y10,25.40,26.71,26.76,26.24,25.55,24.80
1,rolling_scale0.2_nslice3_v1.7_10yrs,y10,25.39,26.71,26.77,26.26,25.57,24.80
2,rolling_scale1.0_nslice3_v1.7_10yrs,y10,25.40,26.71,26.76,26.24,25.55,24.80
3,rolling_scale0.9_nslice2_v1.7_10yrs,y10,25.40,26.72,26.76,26.24,25.55,24.81
4,rolling_scale0.6_nslice3_v1.7_10yrs,y10,25.40,26.72,26.77,26.25,25.56,24.80
...,...,...,...,...,...,...,...,...
291,cadence_drive_gl100_gcbv1.7_10yrs,y1,24.18,25.45,25.52,25.02,24.34,23.74
292,cadence_drive_gl100v1.7_10yrs,y1,24.17,25.45,25.53,25.01,24.34,23.75
293,cadence_drive_gl30v1.7_10yrs,y1,24.16,25.45,25.54,25.03,24.35,23.76
294,cadence_drive_gl200_gcbv1.7_10yrs,y1,24.14,25.52,25.56,24.99,24.31,23.73


In [19]:
# depth stddev data
stddev_data = pd.DataFrame()
for i, yr_tag in enumerate( depth_stddev_all_yrs ):
    depth_stddev_all_yrs[yr_tag] = dbs_order.merge(depth_stddev_all_yrs[yr_tag], on='dbname')
    
    if i == 0:
        stddev_data = depth_stddev_all_yrs[yr_tag]
    else:
        # stack them up
        stddev_data = stddev_data.append(depth_stddev_all_yrs[yr_tag])
        
stddev_data.reset_index(inplace=True, drop=True)

stddev_data

,dbname,yr,u-band coadded dust-corrected depth stddev in eg-footprint,g-band coadded dust-corrected depth stddev in eg-footprint,r-band coadded dust-corrected depth stddev in eg-footprint,i-band coadded dust-corrected depth stddev in eg-footprint,z-band coadded dust-corrected depth stddev in eg-footprint,y-band coadded dust-corrected depth stddev in eg-footprint
0,rolling_scale0.9_nslice3_v1.7_10yrs,y10,0.23,0.19,0.15,0.11,0.11,0.11
1,rolling_scale0.2_nslice3_v1.7_10yrs,y10,0.24,0.19,0.15,0.12,0.11,0.11
2,rolling_scale1.0_nslice3_v1.7_10yrs,y10,0.23,0.19,0.15,0.11,0.11,0.11
3,rolling_scale0.9_nslice2_v1.7_10yrs,y10,0.23,0.19,0.15,0.12,0.11,0.11
4,rolling_scale0.6_nslice3_v1.7_10yrs,y10,0.23,0.19,0.15,0.12,0.11,0.11
...,...,...,...,...,...,...,...,...
291,cadence_drive_gl100_gcbv1.7_10yrs,y1,0.33,0.29,0.17,0.16,0.18,0.16
292,cadence_drive_gl100v1.7_10yrs,y1,0.32,0.31,0.18,0.16,0.17,0.15
293,cadence_drive_gl30v1.7_10yrs,y1,0.34,0.30,0.17,0.16,0.17,0.16
294,cadence_drive_gl200_gcbv1.7_10yrs,y1,0.32,0.27,0.16,0.17,0.17,0.15


#### plot things out to see


In [20]:
baseline_key = 'baseline_nexp2_v1.7_10yrs'

In [21]:
ndbs = len(np.where(depth_data['yr'].values == 'y10')[0])
print(ndbs)

for yr_tag in yr_tags:
    baseline_ind = np.where( (depth_data['yr'].values[:] == yr_tag) & (depth_data['dbname'].values[:] == baseline_key) )[0]
    plt.clf()
    for band in bands:
        ind = np.where(depth_data['yr'].values == yr_tag)[0]
        band_key = '%s-band median coadded dust-corrected depth in eg-footprint' % band
        p = plt.plot(depth_data[band_key].values[ind], range(ndbs), '.-', label='%s-band' % band, )
        plt.plot( [ depth_data[band_key].values[baseline_ind] ] * ndbs, range(ndbs), '--', color=p[0].get_color(), )

    plt.plot( [depth_data[band_key].values[baseline_ind]], [0], 'k--', label='respective median depth for %s' % baseline_key)

    ax = plt.gca()
    ax.set_yticks( range(ndbs) )
    ax.set_yticklabels( dbs_order['dbname'].values )

    plt.xlabel('median depth (coadded; dust extinguished)')
    plt.legend(bbox_to_anchor=(1,1))
    plt.title('%s median depth in eg-footprint' % yr_tag)
    plt.gcf().set_size_inches(10, 30 )
    fname = 'plot_median-depths_%s.png' % yr_tag
    plt.savefig('%s/%s' % (outdir, fname), bbox_inches='tight', format='png')
    print('## saved %s' % fname)
    plt.close() #plt.show()

74
## saved plot_median-depths_y1.png
## saved plot_median-depths_y3.png
## saved plot_median-depths_y6.png
## saved plot_median-depths_y10.png


In [22]:
ndbs = len(np.where(stddev_data['yr'].values == 'y10')[0])

for yr_tag in yr_tags:
    baseline_ind = np.where( (stddev_data['yr'].values[:] == yr_tag) & (stddev_data['dbname'].values[:] == baseline_key) )[0]
    
    plt.clf()
    for band in bands:
        ind = np.where(stddev_data['yr'].values == yr_tag)[0]
        band_key = '%s-band coadded dust-corrected depth stddev in eg-footprint' % band
        p = plt.plot(stddev_data[band_key].values[ind], range(ndbs), '.-', label='%s-band' % band, )
        plt.plot( [ stddev_data[band_key].values[baseline_ind] ] * ndbs, range(ndbs), '--', color=p[0].get_color(), )

    plt.plot( [stddev_data[band_key].values[baseline_ind]], [0], 'k--', label='respective depth stddev for %s' % baseline_key)

    ax = plt.gca()
    ax.set_yticks( range(ndbs) )
    ax.set_yticklabels( dbs_order['dbname'].values )

    plt.xlabel('depth stddev (coadded; dust extinguished)')
    plt.legend(bbox_to_anchor=(1,1))
    plt.title('%s depth stddev in eg-footprint' % yr_tag)
    plt.gcf().set_size_inches(10, 30 )
    fname = 'plot_depth-stddev_%s.png' % yr_tag
    plt.savefig('%s/%s' % (outdir, fname), bbox_inches='tight', format='png')
    print('## saved %s' % fname)
    plt.close() #show()

## saved plot_depth-stddev_y1.png
## saved plot_depth-stddev_y3.png
## saved plot_depth-stddev_y6.png
## saved plot_depth-stddev_y10.png


### save the data

#### for melissa g.

#### for oswg sheet

In [23]:
summary_data

,name,median Y1 $i$-band coadded depth in effective survey area,median Y3 $i$-band coadded depth in effective survey area,median Y6 $i$-band coadded depth in effective survey area,median Y10 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area,std in Y3 $i$-band coadded depth in effective survey area,std in Y6 $i$-band coadded depth in effective survey area,std in Y10 $i$-band coadded depth in effective survey area,Y1 effective survey area,Y3 effective survey area,Y6 effective survey area,Y10 effective survey area
0,rolling_scale0.9_nslice3_v1.7_10yrs,25.03,25.65,25.98,26.24,0.17,0.13,0.12,0.11,14779.75,15062.38,15065.69,15079.06
1,rolling_scale0.2_nslice3_v1.7_10yrs,25.03,25.66,25.99,26.26,0.17,0.13,0.12,0.12,14779.75,15298.96,15184.45,15167.56
2,rolling_scale1.0_nslice3_v1.7_10yrs,25.03,25.65,25.98,26.24,0.17,0.13,0.12,0.11,14779.75,14946.19,15061.18,15048.59
3,rolling_scale0.9_nslice2_v1.7_10yrs,25.03,25.64,25.98,26.24,0.17,0.13,0.12,0.12,14779.75,15243.41,15125.33,15080.17
4,rolling_scale0.6_nslice3_v1.7_10yrs,25.03,25.65,25.99,26.25,0.17,0.13,0.12,0.12,14779.75,15224.84,15124.81,15098.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,cadence_drive_gl100_gcbv1.7_10yrs,25.02,25.66,26.01,26.27,0.16,0.13,0.12,0.11,14639.17,15250.07,15193.31,15201.71
70,cadence_drive_gl100v1.7_10yrs,25.01,25.66,26.00,26.27,0.16,0.13,0.12,0.11,14618.87,15245.93,15185.60,15188.75
71,cadence_drive_gl30v1.7_10yrs,25.03,25.67,26.01,26.27,0.16,0.13,0.12,0.11,14617.45,15232.50,15186.23,15191.95
72,cadence_drive_gl200_gcbv1.7_10yrs,24.99,25.62,25.97,26.24,0.17,0.13,0.12,0.12,13883.60,15061.02,15011.50,15003.69


In [24]:
# if db-order is defined already; one df for everything
for i, yr in enumerate( [1, 3, 6, 10] ):
    depth_data_for_this_yr = depth_data.loc[depth_data['yr'] == 'y%s' % yr]
    stddev_data_for_this_yr = stddev_data.loc[stddev_data['yr'] == 'y%s' % yr]
    
    if i == 0:
        df_in_order = pd.DataFrame({'dbname': summary_data['name'].values})
        # find the indices to map the given order with the order in the df
        # find the indices to get the order
        ind = []
        for db in df_in_order['dbname']:
            #print(db)
            ind_temp = np.where( depth_data_for_this_yr['dbname'].values == db.split('.db')[0] )[0]

            if len(ind_temp) == 0:
                print('%s not found in data dict' % db)
                #ind_temp = np.array([ 0 ])

            if ind_temp in ind:
                print('%s ind already in. %s' % (ind_temp, db))
            ind +=  list( ind_temp )

        ind = np.array(ind)
        print(ind)
    
    # okay assmeble the new dfs
    for colname in stddev_data_for_this_yr:
        if colname != 'dbname' and colname != 'yr':
             df_in_order['Y%s_%s' % (yr, colname)] = stddev_data_for_this_yr[colname].values[ind]

    for colname in depth_data_for_this_yr:
        if colname != 'dbname' and colname != 'yr':
             df_in_order['Y%s_%s' % (yr, colname)] = depth_data_for_this_yr[colname].values[ind]
                
fname = 'exgalm5-data_%sbands_y1-3-6-10.csv' % (len(bands))
df_in_order.to_csv('%s/%s' % (outdir, fname), index=False)
df_in_order

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73]


,dbname,Y1_u-band coadded dust-corrected depth stddev in eg-footprint,Y1_g-band coadded dust-corrected depth stddev in eg-footprint,Y1_r-band coadded dust-corrected depth stddev in eg-footprint,Y1_i-band coadded dust-corrected depth stddev in eg-footprint,Y1_z-band coadded dust-corrected depth stddev in eg-footprint,Y1_y-band coadded dust-corrected depth stddev in eg-footprint,Y1_u-band median coadded dust-corrected depth in eg-footprint,Y1_g-band median coadded dust-corrected depth in eg-footprint,Y1_r-band median coadded dust-corrected depth in eg-footprint,...,Y10_r-band coadded dust-corrected depth stddev in eg-footprint,Y10_i-band coadded dust-corrected depth stddev in eg-footprint,Y10_z-band coadded dust-corrected depth stddev in eg-footprint,Y10_y-band coadded dust-corrected depth stddev in eg-footprint,Y10_u-band median coadded dust-corrected depth in eg-footprint,Y10_g-band median coadded dust-corrected depth in eg-footprint,Y10_r-band median coadded dust-corrected depth in eg-footprint,Y10_i-band median coadded dust-corrected depth in eg-footprint,Y10_z-band median coadded dust-corrected depth in eg-footprint,Y10_y-band median coadded dust-corrected depth in eg-footprint
0,rolling_scale0.9_nslice3_v1.7_10yrs,0.36,0.33,0.19,0.17,0.18,0.17,24.17,25.41,25.52,...,0.15,0.11,0.11,0.11,25.40,26.71,26.76,26.24,25.55,24.80
1,rolling_scale0.2_nslice3_v1.7_10yrs,0.36,0.33,0.19,0.17,0.18,0.17,24.17,25.41,25.52,...,0.15,0.12,0.11,0.11,25.39,26.71,26.77,26.26,25.57,24.80
2,rolling_scale1.0_nslice3_v1.7_10yrs,0.36,0.33,0.19,0.17,0.18,0.17,24.17,25.41,25.52,...,0.15,0.11,0.11,0.11,25.40,26.71,26.76,26.24,25.55,24.80
3,rolling_scale0.9_nslice2_v1.7_10yrs,0.36,0.33,0.19,0.17,0.18,0.17,24.17,25.41,25.52,...,0.15,0.12,0.11,0.11,25.40,26.72,26.76,26.24,25.55,24.81
4,rolling_scale0.6_nslice3_v1.7_10yrs,0.36,0.33,0.19,0.17,0.18,0.17,24.17,25.41,25.52,...,0.15,0.12,0.11,0.11,25.40,26.72,26.77,26.25,25.56,24.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,cadence_drive_gl100_gcbv1.7_10yrs,0.33,0.29,0.17,0.16,0.18,0.16,24.18,25.45,25.52,...,0.15,0.11,0.11,0.11,25.38,26.55,26.74,26.27,25.59,24.81
70,cadence_drive_gl100v1.7_10yrs,0.32,0.31,0.18,0.16,0.17,0.15,24.17,25.45,25.53,...,0.15,0.11,0.11,0.11,25.38,26.53,26.73,26.27,25.59,24.81
71,cadence_drive_gl30v1.7_10yrs,0.34,0.30,0.17,0.16,0.17,0.16,24.16,25.45,25.54,...,0.14,0.11,0.10,0.11,25.40,26.63,26.75,26.27,25.58,24.81
72,cadence_drive_gl200_gcbv1.7_10yrs,0.32,0.27,0.16,0.17,0.17,0.15,24.14,25.52,25.56,...,0.15,0.12,0.10,0.09,25.31,26.63,26.69,26.24,25.57,24.82


okay good -- the dbs are in the same order as before and the values for them match between the older file and the newer one

#### extra things

##### check to confirm the new data saved for y10 matches that saved earlier

#### lets plot out the stddev in depth

#### just for a sanity check: lets plot out the area values in each file; they should match.

okay good. the area values in the files are the same as they should be.